In [16]:
import numpy as np
import os
import scipy.io as scio

In [6]:
def process_data(data, n):
    data = np.transpose(data,[2, 0, 1])
    split_lists = [data[i:i+n] for i in range(0, data.shape[0], n)]
    summed_lists = [sum(sub_list) for sub_list in split_lists]
    summed_lists = np.transpose(np.array(summed_lists),[1, 2, 0])
    return summed_lists

In [ ]:
x = np.zeros([1080, 1920, 49])
y = process_data(x, 2)
print(y)

# 若除不尽，则剩余通道会保留，叠加为最后一个通道
# 如49//2 为24channels，剩余一个通道为25channel

In [3]:
# def process_data2(data):
#     # average_data = np.mean(data[:, :, ::2], axis=2)
#     data1 = data[:, :, ::2]  # 偶
#     data2 = data[:, :, 1::2] # 奇

#     return data1 + data2
def process_data3(data):
    # average_data = np.mean(data[:, :, ::2], axis=2)
    data1 = data[:, :, ::3]  
    data2 = data[:, :, 1::3] 
    data3 = data[:, :, 2::3] 
    return data1 + data2 + data3

HeatCubes

In [7]:
matfile = ''
root = '/home/ljm/workspace/zx/HADAR_database/'
out_root = '/mnt/Disk/zx/HADAR/HADAR_database_6'
split_num = 9
for i in list(range(1,12)):
    filename = os.listdir(os.path.join(root, f'Scene{i}', 'HeatCubes'))
    out_filename = os.path.join(out_root, f'Scene{i}', 'HeatCubes')
    if not os.path.exists(out_filename):
        os.makedirs(out_filename)
    # print(filename)
    # print(len(filename))
    for file in filename:
        if file[-3:] == 'mat':
            matfile = os.path.join(root, f'Scene{i}', 'HeatCubes') + '/' + file
            if i == 11:
                data = np.array(scio.loadmat(matfile)["HSI"])
            else:
                data = np.array(scio.loadmat(matfile)["S"])
            data_ = process_data(data, split_num)
            scio.savemat(out_filename + '/' + file, {'S': data_})
            np.save(out_filename + '/' + file[:-3] + 'npy', data_)
            print(out_filename + '/' + file + ' is saved')
            print(out_filename + '/' + file[:-3] + 'npy' + ' is saved')
            

/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/L_0001_heatcube.mat is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/L_0001_heatcube.npy is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/R_0002_heatcube.mat is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/R_0002_heatcube.npy is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/R_0004_heatcube.mat is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/R_0004_heatcube.npy is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/L_0003_heatcube.mat is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/L_0003_heatcube.npy is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/R_0003_heatcube.mat is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/R_0003_heatcube.npy is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/R_0001_heatcube.mat is saved
/mnt/Disk/zx/HADAR/HADAR_database_6/Scene1/HeatCubes/R_0001_heatcube.npy is saved
/mnt/Disk/zx/HAD

emiLib.mat

In [22]:
root = '/home/ljm/workspace/zx/HADAR_database/'
out_root = '/mnt/Disk/zx/HADAR/HADAR_database_6'
split_num = 9

emi_file = os.path.join(root, 'emiLib.mat')
data = np.array(scio.loadmat(emi_file)["matLib"]).astype(np.float32)

data = np.concatenate((data[0:4,:], data), axis=0)
data = np.concatenate((data, np.expand_dims(data[-1,:], axis=0)), axis=0)

split_lists = [data[i:i + split_num] for i in range(0, data.shape[0], split_num)]
summed_lists = [sum(sub_list)/split_num for sub_list in split_lists]
summed_lists = np.array(summed_lists)

# 通道数不够则直接赋相邻值  其实相差不是很大
# head = np.expand_dims(summed_lists[0], axis=0)
# summed_lists = np.concatenate((head, summed_lists), axis = 0)
# tail = np.expand_dims(summed_lists[-1], axis=0)
# summed_lists = np.concatenate((summed_lists, tail), axis = 0)

print(summed_lists.shape)
scio.savemat(out_root + '/emiLib_.mat', {'matLib': summed_lists})
print(out_root + '/emiLib_.mat' + ' is saved')


(6, 30)
/mnt/Disk/zx/HADAR/HADAR_database_6/emiLib_.mat is saved


### .mat to .npy
读取.mat

In [5]:
import time
file = '/home/ljm/workspace/zx/HADAR_database/Scene1/HeatCubes/L_0001_heatcube.mat'
mat_start_time = time.time()
mat = scio.loadmat(file)["S"]
mat_end_time = time.time()
print(mat_end_time - mat_start_time)


1.5963153839111328


保存.npy

In [9]:
save_dir = '/home/ljm/workspace/zx/HADAR_database1/Scene1/HeatCubes/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
np.save(os.path.join(save_dir, 'L_0001_heatcube.npy'), mat)

In [11]:
npy_start_time = time.time()
npy = np.load(os.path.join(save_dir, 'L_0001_heatcube.npy'))
npy_end_time = time.time()
print(npy_end_time - npy_start_time)

0.2799663543701172


In [12]:
npy == mat

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  Tr